## **Setup**

In [1]:
!pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

## **Store Part I results into DataFrame**
* Load the csv exported in Part I to a DataFrame

In [4]:
file = "output_data/cities.csv"

cities_data = pd.read_csv(file)

cities_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,atuona,-9.80,-139.03,79.20,78,15,19.73,PF,1598746346
1,talnakh,69.49,88.40,44.60,100,90,4.47,RU,1598746346
2,white rock,35.25,-91.73,75.99,94,1,9.62,US,1598746222
3,puerto quijarro,36.59,-6.23,68.00,45,8,4.70,ES,1598746346
4,port alfred,-7.03,29.76,75.07,52,0,5.61,CD,1598746163



## **Humidity Heatmap**
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key
gmaps.configure(g_key)

In [6]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_data[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
#census_data_complete = census_data_complete.dropna()
humidity = cities_data["Humidity"].astype(float)

In [7]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## **Create new DataFrame fitting weather criteria**
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
weather_cond = cities_data.loc[(cities_data["Max Temp"] > 70) & (cities_data["Max Temp"] < 80) & (cities_data["Wind Speed"] < 10) & (cities_data["Cloudiness"] == 0), :]

weather_cond

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,port alfred,-7.03,29.76,75.07,52,0,5.61,CD,1598746163
100,arraial do cabo,-22.84,-46.65,78.80,36,0,3.36,BR,1598746370
153,korla,41.76,86.15,71.01,66,0,3.00,CN,1598746381
187,maua,-23.67,-46.46,75.20,50,0,3.36,BR,1598746389
222,port lincoln,-7.03,29.76,75.07,52,0,5.61,CD,1598746163
232,port elizabeth,-7.03,29.76,75.07,52,0,5.61,CD,1598746163
264,port hardy,-7.03,29.76,75.07,52,0,5.61,CD,1598746407
281,port hedland,-7.03,29.76,75.07,52,0,5.61,CD,1598746163
305,port hawkesbury,-7.03,29.76,75.07,52,0,5.61,CD,1598746415
326,agdam,39.99,46.93,70.02,57,0,3.02,AZ,1598746420


In [9]:
ideal_weather = weather_cond.dropna()

## **Hotel Map**
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
#Add hotel name
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ""

#Define target
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

index_1 = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # request using dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[index_1, -1] = name
    index_1 = index_1 + 1

hotel_df = hotel_df.dropna()

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,port alfred,-7.03,29.76,75.07,52,0,5.61,CD,1598746163,
100,arraial do cabo,-22.84,-46.65,78.80,36,0,3.36,BR,1598746370,
153,korla,41.76,86.15,71.01,66,0,3.00,CN,1598746381,
187,maua,-23.67,-46.46,75.20,50,0,3.36,BR,1598746389,
222,port lincoln,-7.03,29.76,75.07,52,0,5.61,CD,1598746163,
232,port elizabeth,-7.03,29.76,75.07,52,0,5.61,CD,1598746163,
264,port hardy,-7.03,29.76,75.07,52,0,5.61,CD,1598746407,
281,port hedland,-7.03,29.76,75.07,52,0,5.61,CD,1598746163,
305,port hawkesbury,-7.03,29.76,75.07,52,0,5.61,CD,1598746415,
326,agdam,39.99,46.93,70.02,57,0,3.02,AZ,1598746420,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add layer

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))